<a href="https://colab.research.google.com/github/DataScience-in-Tourism/Chapter-15-Introduction-Natural-Language-Processing/blob/main/Text_cleaning_Tourism_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
# Dataset will be stored in a Pandas Dataframe

url = 'https://raw.githubusercontent.com/DataScience-in-Tourism/Chapter-15-Introduction-Natural-Language-Processing/main/travelsomeday.csv'
df = pd.read_csv(url)
df.head(1)

# EDITTED

,,,,,,,,,,,,,,,,,,,id,guid,link,pubdate,author,title,description,source,user_id,profile_image_url,user_statuses_count,user_friends_count,user_followers_count,user_created_at,user_bio,user_location,lang,coords,Unnamed: 18
1,https://twitter.com/_LoveCaribbean/statuses/1252926931634458624,https://twitter.com/_LoveCaribbean/statuses/1252926931634458624,2020-04-22 07:47:13,_LoveCaribbean,RT @grenadaexplorer: A tropical treasure to cherish forever 🌅 For exclusive #TravelTips and island insights visit https://t.co/wFawu512ad #…,A tropical treasure to cherish forever 🌅 For exclusive #TravelTips and island insights visit https://t.co/wFawu512ad #Grenada #TimeToLime #PureGrenada #SpiceIsle #ILoveGrenada #DreamTodayTravelTomorrow #TravelSomeDay #WeBelieveInTravel #GrenadaDreaming #CaribbeanDreaming https://t.co/64Kg2mPH3l,Twitter Web App,1356684936,https://pbs.twimg.com/profile_images/474150473876770817/aNbE9lsf_normal.jpeg,8047,1099,3019,2013-04-16 07:55:23,#CaribbeanDreaming The diversity of the Caribbean makes it a wonderful holiday destination whatever your tastes or interests.,London,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As we can see from here, the text data is at the index. We need to convert indexes to columns.

In order to do this, first we need to give a name to columns in the index

In [10]:
df= df.rename_axis(['id1','url','url2','date','username', 'quote', 'tweet','as1','as2','as3','as4','as5','as6','as7','as8','as9','as10','as11','a12'])
df.columns.to_list()

['id',
 'guid',
 'link',
 'pubdate',
 'author',
 'title',
 'description',
 'source',
 'user_id',
 'profile_image_url',
 'user_statuses_count',
 'user_friends_count',
 'user_followers_count',
 'user_created_at',
 'user_bio',
 'user_location',
 'lang',
 'coords',
 'Unnamed: 18']

In [11]:
df.sample(1)

,,,,,,,,,,,,,,,,,,,id,guid,link,pubdate,author,title,description,source,user_id,profile_image_url,user_statuses_count,user_friends_count,user_followers_count,user_created_at,user_bio,user_location,lang,coords,Unnamed: 18
id1,url,url2,date,username,quote,tweet,as1,as2,as3,as4,as5,as6,as7,as8,as9,as10,as11,a12,,,,,,,,,,,,,,,,,,,
35,https://twitter.com/SailingGrenada/statuses/1252839567830298624,https://twitter.com/SailingGrenada/statuses/1252839567830298624,2020-04-22 02:00:04,SailingGrenada,An afternoon exploring in #Grenada. Visit https://t.co/suQM7MXLLR for top #SailingTips ☀️⛵️🏝️ #Sailing #PureGrenada #ILoveGrenada #DreamTodayTravelTomorrow #TravelSomeDay #WeBelieveInTravel #GrenadaDreaming https://t.co/h1Q5h7Yova,An afternoon exploring in #Grenada. Visit https://t.co/suQM7MXLLR for top #SailingTips ☀️⛵️🏝️ #Sailing #PureGrenada #ILoveGrenada #DreamTodayTravelTomorrow #TravelSomeDay #WeBelieveInTravel #GrenadaDreaming https://t.co/h1Q5h7Yova,Buffer,232822620,https://pbs.twimg.com/profile_images/557553781584322562/YnZwFtM3_normal.jpeg,4210,617,1576,2011-01-01 07:09:32,Royal Yachting Association (RYA) training centre based in Grenada... The BEST Sailing Destination in the Caribbean!,Grenada & the Grenadines,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.reset_index(inplace=True)
df.sample(1)

,id1,url,url2,date,username,quote,tweet,as1,as2,as3,as4,as5,as6,as7,as8,as9,as10,as11,a12,id,guid,link,pubdate,author,title,description,source,user_id,profile_image_url,user_statuses_count,user_friends_count,user_followers_count,user_created_at,user_bio,user_location,lang,coords,Unnamed: 18
2988,2989,https://twitter.com/travelsewhere/statuses/125...,https://twitter.com/travelsewhere/statuses/125...,2020-05-11 12:10:11,travelsewhere,"What to See On a Day Trip to Skofja Loka, Slov...","What to See On a Day Trip to Skofja Loka, Slov...",Crowdfire App,3310784164,https://pbs.twimg.com/profile_images/795385202...,6724,3173,7563,2015-06-06 11:36:17,"David, Australian #travelblogger who\'s always...",Europe,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.tweet.sample(5).to_list()

['The Best Budget Destinations to Add to your Bucket List #travelsomeday #armchairtravel #staythefhome https://t.co/U95nRurlVl',
 'CDC announces no cruising through July 19: https://t.co/BD83s859He #TravelSomeday #Coronavirus',
 'Watching #Shakespeare at the Rose Theatre #York #travelsomeday #armchairtravel #plannowtravellater https://t.co/oaBFaa3GRN #england',
 'How to Travel for Free by #Volunteering Abroad #travelsomeday #armchairtravel #staythefhome https://t.co/8vgMkEaIFy #traveltips',
 'Better than Xanax https://t.co/giiCLTHwFO #TravelSomeday']

In [14]:
#We have two columns as a text. Let's check them and decide which one we will work on
df.loc[:,['quote', 'tweet']].sample(10)

,quote,tweet
5860,Japan is an amazing country and I can\'t wait ...,Japan is an amazing country and I can\'t wait ...
3750,RT @SailingGrenada: Why is it a good idea to k...,Why is it a good idea to keep a logbook? ⛵️ 📙 ...
1240,"Hudson Yards, New York #TravelSomeday https://...","Hudson Yards, New York #TravelSomeday https://..."
3695,Trento: The Loveliest Italian City You’ve Neve...,Trento: The Loveliest Italian City You’ve Neve...
289,Here are some surprisingly cool destinations -...,Here are some surprisingly cool destinations -...
846,RT @endlessadventu: Ready to plan you next vac...,Ready to plan you next vacation? ✈️Getting tir...
4552,Breaking News: #Fastpasses will be suspended a...,Breaking News: #Fastpasses will be suspended a...
2441,World Map #Christmas Gift Ideas to Inspire #Wa...,World Map #Christmas Gift Ideas to Inspire #Wa...
2966,The Best Hostel in Barcelona – Fabrizzio’s Pet...,The Best Hostel in Barcelona – Fabrizzio’s Pet...
5551,RT @clairesturz: #Hiking the Cocora Valley in ...,#Hiking the Cocora Valley in #Colombia #travel...


It looks like quote column is mostly retweets. I will go with ***tweet*** column from now on, and apply text cleaning on it.

# **Text Cleaning**

In this step, these text cleaning steps will be applied on tweets:



*   Make all text lower case
*   Language Detection
*   Expanding Contractions
*   Removal of Stop Words
*   Removing URLs
*   Remove html HTML tags
*   Removing Emojis
*   Remove Emoticons
*   Stemming & Lemmatization (and the difference)
*   Spell Correction
*   Normalization
*   Removing Punctuation
*   TF-IDF



In [15]:
!pip install -q wordcloud
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# **Feature Extraction**

In [16]:
# Here are the feature enginnering scripts that can be done before data cleaning

# Number of stopwords
stop = stopwords.words('english')
df['stopwords'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))

# Number of punctuation
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['tweet'].apply(lambda x: count_punct(x))

# Number of hashtag characters
df['hastags'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))

# Number of numerics
df['numerics'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

# Number of Uppercase words
df['upper'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

# sentence count
df['sentence_count'] = df["tweet"].apply(lambda x: len(str(x).split(".")))


In [17]:
df.head(5)

,id1,url,url2,date,username,quote,tweet,as1,as2,as3,as4,as5,as6,as7,as8,as9,as10,as11,a12,id,guid,link,pubdate,author,title,description,source,user_id,profile_image_url,user_statuses_count,user_friends_count,user_followers_count,user_created_at,user_bio,user_location,lang,coords,Unnamed: 18,stopwords,punctuation,hastags,numerics,upper,sentence_count
0,1,https://twitter.com/_LoveCaribbean/statuses/12...,https://twitter.com/_LoveCaribbean/statuses/12...,2020-04-22 07:47:13,_LoveCaribbean,RT @grenadaexplorer: A tropical treasure to ch...,A tropical treasure to cherish forever 🌅 For e...,Twitter Web App,1356684936,https://pbs.twimg.com/profile_images/474150473...,8047,1099,3019,2013-04-16 07:55:23,#CaribbeanDreaming The diversity of the Caribb...,London,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,21,11,0,1,3
1,2,https://twitter.com/carolcruising/statuses/125...,https://twitter.com/carolcruising/statuses/125...,2020-04-22 07:44:47,carolcruising,RT @clairesturz: Backpacking in #Cuba on a Bud...,Backpacking in #Cuba on a Budget #travelsomeda...,Twitter Web App,319114596,https://pbs.twimg.com/profile_images/630068953...,26683,660,1227,2011-06-17 11:47:16,Carol Colborn is the author of Carolina: Cruis...,"Phoenix, AZ",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,9,4,0,0,2
2,3,https://twitter.com/Frommers/statuses/12529251...,https://twitter.com/Frommers/statuses/12529251...,2020-04-22 07:40:04,Frommers,We likeee! https://t.co/2g1HMCpsVn #DeltaAirl...,We likeee! https://t.co/2g1HMCpsVn #DeltaAirl...,Hootsuite Inc.,66244710,https://pbs.twimg.com/profile_images/463423099...,69152,32598,153703,2009-08-16 21:44:37,The Most Trusted Name In Travel since 1957.,Everywhere,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,9,3,0,0,2
3,4,https://twitter.com/clairesturz/statuses/12529...,https://twitter.com/clairesturz/statuses/12529...,2020-04-22 07:32:39,clairesturz,Experience Buenos Aires like a local #travelso...,Experience Buenos Aires like a local #travelso...,Revive Social App,874023301,https://pbs.twimg.com/profile_images/613107654...,39950,19520,24799,2012-10-11 13:32:59,Helping you plan your best future travel adven...,"England, United Kingdom",es,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,9,4,0,0,2
4,5,https://twitter.com/grenadaexplorer/statuses/1...,https://twitter.com/grenadaexplorer/statuses/1...,2020-04-22 07:32:02,grenadaexplorer,Who can\'t wait for Grenada Sailing Festival 2...,Who can\'t wait for Grenada Sailing Festival 2...,Buffer,17083471,https://pbs.twimg.com/profile_images/703782166...,4070,856,1882,2008-10-31 07:22:26,Travel & Island Guide for Grenada in the Carib...,Grenada,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,25,11,0,0,3


# Text Cleaning

### **Make all text lower case**

In [18]:
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['tweet'].head()

0    a tropical treasure to cherish forever 🌅 for e...
1    backpacking in #cuba on a budget #travelsomeda...
2    we likeee! https://t.co/2g1hmcpsvn #deltaairli...
3    experience buenos aires like a local #travelso...
4    who can\'t wait for grenada sailing festival 2...
Name: tweet, dtype: object

### **Language Detection**


In [ ]:
!pip install umap-learn
import os, re, nltk, spacy, string, umap


pd.set_option('display.max_colwidth', 50) 
nlp = spacy.load('en_core_web_sm')

!pip install spacy-langdetect
from spacy_langdetect import LanguageDetector
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

     |████████████████████████████████| 81kB 5.2MB/s 
     |████████████████████████████████| 1.2MB 22.0MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=6f31d53ff124901b271f184c49a9c7ab8f255de708ea88f5fd20af4d1d36ccff
  Stored in directory: /root/.cache/pip/wheels/ad/df/d5/a3691296ff779f25cd1cf415a3af954b987fb53111e3392cf4
  Created wheel for pynndescent: filename=pynndescent-0.5.2-cp37-none-any.whl size=51351 sha256=32c65e1a6d0fd20f8652def787024b3c8737ea5a5b0de2306cc91b32341193e4
  Stored in directory: /root/.cache/pip/wheels/ba/52/4e/4c28d04d144a28f89e2575fb63628df6e6d49b56c5ddd0c74e
Successfully built umap-learn pynndescent
     |████████████████████████████████| 1.0MB 14.4MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp37-none-any.whl size=993460 sha256=8c194d3fd8e3c43f79d0151603f5324603255762e2a31a92c6b041c713e14828
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7

In [ ]:
def detect_language(tweet):
    try:
        doc = nlp(t)
        language = doc._.language['language']
        score = doc._.language['score']
    except:
        language = ''
        print('sth goes wrong')
    return language, score

languages = []
scores = []
for i, t in df['tweet'].iteritems():
    l, s = detect_language(t)
    if i % 500 ==0:   # depending on the size of your data, you can change this number
        print(i, t, l)  
    languages.append(l)
    scores.append(s)
    
df['lng'] = languages
df['lng_score'] = scores

0 a tropical treasure to cherish forever 🌅 for exclusive #traveltips and island insights visit https://t.co/wfawu512ad #grenada #timetolime #puregrenada #spiceisle #ilovegrenada #dreamtodaytraveltomorrow #travelsomeday #webelieveintravel #grenadadreaming #caribbeandreaming https://t.co/64kg2mph3l en
500 sometimes sesha is draconine, othertimes he looks like this. 🇹🇭 @thailandinsider @thailandfanclub @gothaibefree #thailand #bangkok #watprakaew #ww #tt #ttot #travel #travelblogger #solotravel #gaytravel @ilovelgbttravel #travelfromhome #travelsomeday #armchairtravel #picoftheday https://t.co/llfjacahsf en
1000 happy #earthday everyone! here are a few of my favorite photos which celebrate the natural wonders of earth! share your favorite photos that celebrate this great planet! 🌟🌟🌟🌟🌟 #earthday2020 #wednesdaymorning #stayhome #travelsomeday #wednesdaymotivation https://t.co/wlczwhaqmm en
1500 the only #coronavirus change that we hope will stick around https://t.co/2g1hmcpsvn #travelsomeda

In [ ]:
df[['tweet', 'lng','lng_score']].sample(10).round(2)

,tweet,lng,lng_score
5640,exploring catalunya: an #adventure at campdevá...,en,1.00
18,mont joli\'s interesting take on a military su...,en,0.43
385,"love that ""not imposing at all"" feng shui goin...",en,1.00
3047,how to manage the altitude in #cusco and altit...,en,1.00
169,why staying in a hostel for the first time isn...,en,1.00
2063,where to stay in brno – internesto apartments ...,en,1.00
1905,"it\'s been a while since i went to the states,...",en,1.00
2725,oír roncar a mi marido y al vecino de arriba a...,es,1.00
3114,why staying in a hostel for the first time isn...,en,1.00
4593,"well, that’s new one. 😏 @experiencegr #experie...",en,1.00


In [ ]:
df.lng.value_counts()

en    5888
no      72
it      19
fr      12
es       9
sv       9
da       6
ca       4
af       2
sl       1
nl       1
hi       1
el       1
id       1
ro       1
Name: lng, dtype: int64

In [ ]:
language_counts = df.lng.value_counts()
pd.DataFrame(language_counts)

,lng
en,5888
no,72
it,19
fr,12
es,9
sv,9
da,6
ca,4
af,2
sl,1


'no' stands for Norwegian language. 

For more info about acronyms of languages:
https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes

In [ ]:
# Number of English Tweets
df.loc[df.lng == 'en'].shape

(5888, 46)

In [ ]:
# Keep only English language and drop the rest
df = df.loc[df.lng == 'en']

### **Expand Contractions**

Contractions are words or combinations of words which are shortened by dropping letters and replacing them by an apostrophe. 

Let’s have a look at some examples:
* we’re = we are
* we’ve = we have
* I’d = I would

Note: This step needs to be done before word tokenizer because NLTK word tokenizer has in-built methods for dealing with contractions. However, NLTK word tokenizer's approach separates contractions without expanding. Expanding is a better method than simply separating.

In [19]:
!pip install contractions

     |████████████████████████████████| 327kB 23.9MB/s 
     |████████████████████████████████| 286kB 34.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85398 sha256=7ddf615c92a5a57ac07b41dcf925aa736d57e13c5556c62fc675d17ae2a49753
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [20]:
import contractions

# Example text 
text = ''' She'll be airport in 30 mins. We are supposed to catch the arrival, aren't we?  
          I'd love to welcome her personally. It'll be an awesome vacation.'''
  
# creating an empty list 
expanded_words = []     
for word in text.split(): 
  # using contractions.fix to expand the shotened words 
  expanded_words.append(contractions.fix(word))    
    
expanded_text = ' '.join(expanded_words) 
print('Original text: ' + text) 
print('\n') 
print('Expanded_text: ' + expanded_text)

Original text:  She'll be airport in 30 mins. We are supposed to catch the arrival, aren't we?  
          I'd love to welcome her personally. It'll be an awesome vacation.


Expanded_text: she will be airport in 30 mins. We are supposed to catch the arrival, are not we? I would love to welcome her personally. it will be an awesome vacation.


In [21]:
df['tweet'] = df['tweet'].apply(lambda x: contractions.fix(x))
df.tweet.sample(5)

3698    the essential byron bay #backpacking guide #tr...
415     wondering how you can help the travel industry...
1730    #djurönäset – relax in the #stockholm archipel...
1729    #sicily wants to subsidize your next vacation ...
3639    the essential byron bay #backpacking guide #tr...
Name: tweet, dtype: object

In [45]:
df.tweet.head()

0    a tropical treasure to cherish forever 🌅 for e...
1    backpacking in #cuba on a budget #travelsomeda...
2    we likeee! https://t.co/2g1hmcpsvn #deltaairli...
3    experience buenos aires like a local #travelso...
4    who can\'t wait for grenada sailing festival 2...
Name: tweet, dtype: object

### **Removal of Stop Words** 

In [47]:
df['stopwords'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['tweet','stopwords']].head()

,tweet,stopwords
0,a tropical treasure to cherish forever 🌅 for e...,4
1,backpacking in #cuba on a budget #travelsomeda...,3
2,we likeee! https://t.co/2g1hmcpsvn #deltaairli...,1
3,experience buenos aires like a local #travelso...,1
4,who can\'t wait for grenada sailing festival 2...,3


In [ ]:
# before removing stopwords, we can count count them.
# Remember that we already did this in the beginning. But it could be done here as well.
df['stopwords'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['tweet','stopwords']].head()

,tweet,stopwords
0,a tropical treasure to cherish forever 🌅 for e...,4
1,backpacking in #cuba on a budget #travelsomeda...,3
2,we likeee! https://t.co/2g1hmcpsvn #deltaairli...,1
3,experience buenos aires like a local #travelso...,1
4,who can\'t wait for grenada sailing festival 2...,3


In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['tweet'].sample(10)

5009    coming home, reverse culture shock now? #trave...
249     backpacking bogota: things #bogota #colombia #...
3037    long weekend #riga #latvia #travelsomeday #arm...
2364    #hobbitenango – #hobbits guatemala’s middle ea...
5223    travel alone first time #armchairtravel #trave...
3444    know it\'s law wear high heels acropolis? rhod...
3462    #pisac – exploring peru’s sacred valley #trave...
3569    perfect #cusco itinerary: spend 2 days cusco #...
4044    rules govern dining #spain https://t.co/3crlpb...
5876    48 hours brno, czech republic #travelsomeday #...
Name: tweet, dtype: object

#### **Adding common words from our document to stop_words**

In [ ]:
add_words = ["also","im","ive",]

stop_words = set(stopwords.words("english"))
stop_added = stop_words.union(add_words)

df['tweet'] = df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_added))
df['tweet'].sample(10)

5843    taste #belize food tours: best belize culture ...
3690    one thrilling things #caribbean #island charac...
5598    castles sintra: fairytale palaces, gardens his...
5166    #trek machu picchu #travelsomeday #armchairtra...
4758    in: https://t.co/7ydcjbztkw #coronavirus #trav...
1952    go many multigenerational breaks? abergavenny ...
697     airbnb first time #discount code: $40 #airbnb ...
3751    know it\'s law wear high heels acropolis? rhod...
1858    a66 want go back california, 20 years can\'t w...
1757    ever see tongue #bumblebee? 🐝 @visithudsonny #...
Name: tweet, dtype: object

### **Remove URLs** 

In [ ]:
import re

def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

df['tweet'] = df.tweet.apply(remove_urls)
df.tweet.head()

0    tropical treasure cherish forever 🌅 exclusive ...
1    backpacking #cuba budget #travelsomeday #armch...
3    experience buenos aires like local #travelsome...
4    can\'t wait grenada sailing festival 2021?! ☝️...
5    best hostel barcelona – fabrizzio’s petit revi...
Name: tweet, dtype: object

In [ ]:
# Example
sentence=  'NPR can provide useful information https://www.npr.org'
remove_urls(sentence)

'NPR can provide useful information '

### **Remove html tags** 

In [ ]:
from bs4 import BeautifulSoup

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

#Example
strip_html_tags('People and neighbors are for the most part very friendly,</a> even in stores</td>')

'People and neighbors are for the most part very friendly, even in stores'

In [ ]:
df['tweet'] = df['tweet'].apply(lambda x: strip_html_tags(x))
df['tweet'].head()

0    tropical treasure cherish forever 🌅 exclusive ...
1    backpacking #cuba budget #travelsomeday #armch...
3    experience buenos aires like local #travelsome...
4    can\'t wait grenada sailing festival 2021?! ☝️...
5    best hostel barcelona – fabrizzio’s petit revi...
Name: tweet, dtype: object

###**Removing Emojis**

Eventhough emojis can be indictor of some emotions that can be related to the tweet, we need to remove the emojis in our text analysis because for now we don't have a strong method to utulize emojis.

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

#Example
remove_emoji("Have fun with NLP! 😃😃")

'Have fun with NLP! '

In [ ]:
# remove all emojis from tweets
df['tweet'] = df['tweet'].apply(lambda x: remove_emoji(x))

### **Remove Emoticons**

In previous steps, we have removed emoji. Now, going to remove emoticons.

What is the difference between emoji and emoticons?

:-) is an emoticon
😜 → emoji.

In [ ]:
!pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Function for removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

In [ ]:
#Example
remove_emoticons("I used to play Super Mario games :)")

'I used to play Super Mario games '

In [ ]:
df['tweet'] = df['tweet'].apply(lambda x: remove_emoticons(x))
df.tweet.sample(5)

3715    death road #bolivia – world’s dangerous road #...
844     pauline frommer (author frommer guides #nyc wr...
3014    watching vivid grand show palast #berlin #trav...
2006    10 handy travel apps solve problems #travelsom...
1114    happy #earthday everyone! favorite photos cele...
Name: tweet, dtype: object

### **Stemming and Lemmatization**

**Definition**: Stemming usually refers to a crude heuristic process that chops off the ends of words in the hope of achieving this goal correctly most of the time, and often includes the removal of derivational affixes. Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma . 

Source: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html



### **Tokenization**

During stemming or lemmatization process, tokenization will also be applied.

For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words.

There are different types of tokenization. For further info, you can check these articles:

> https://towardsdatascience.com/an-introduction-to-tweettokenizer-for-processing-tweets-9879389f8fe7

> https://towardsdatascience.com/benchmarking-python-nlp-tokenizers-3ac4735100c5





In [ ]:
#Example (adapted from datacamp)
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
porter=PorterStemmer()

In [ ]:
#Tourism specific words to be stemmed and lemmatized
word_list = ["tourist", "booking", "rating","itinerary","recreation","amenities","attractions","sightseeing","eating"]
print("{0:20}{1:20}{2:20}".format("Original Word","Stemmed","Lemmatized"))

for word in word_list:
    print("{0:20}{1:20}{2:20}".format(word,porter.stem(word),wordnet_lemmatizer.lemmatize(word,pos="v")))

Original Word       Stemmed             Lemmatized          
tourist             tourist             tourist             
booking             book                book                
rating              rate                rat                 
itinerary           itinerari           itinerary           
recreation          recreat             recreation          
amenities           amen                amenities           
attractions         attract             attractions         
sightseeing         sightse             sightsee            
eating              eat                 eat                 


In [ ]:
#We can apply just one of the stemming or lemmatizion. For this study, we will go with lemmatization.
#Still here is the codes for stemming in case you want to use stemming instead of lemmatization
'''
porter=PorterStemmer()

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

df['tweet'] = df['tweet'].apply(lambda x: stemSentence(x))
df.sample(5)
'''

'\nporter=PorterStemmer()\n\ndef stemSentence(sentence):\n    token_words=word_tokenize(sentence)\n    token_words\n    stem_sentence=[]\n    for word in token_words:\n        stem_sentence.append(porter.stem(word))\n        stem_sentence.append(" ")\n    return "".join(stem_sentence)\n\ndf[\'tweet\'] = df[\'tweet\'].apply(lambda x: stemSentence(x))\ndf.sample(5)\n'

In [ ]:
# Lemmatization and Tokenization

def LemmaSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    lemma_sentence=[]
    for word in token_words:
        lemma_sentence.append(wordnet_lemmatizer.lemmatize(word, pos= 'v'))
        lemma_sentence.append(" ")
    return "".join(lemma_sentence)

df['tweet'] = df['tweet'].apply(lambda x: LemmaSentence(x))
df.tweet.sample(5)

671     make friends new city # travelsomeday # armcha...
2098    enter # colombia emergency # passport # travel...
2143    free things phoenix , # arizona # travelsomeda...
4260    # copan – mayan ruin & macaws # honduras # tra...
301     study spanish costa rica one week # travelsome...
Name: tweet, dtype: object

In [ ]:
df.tweet.dtype

dtype('O')

### **Spell Correction** 
*Reminder: below explanation was taken from somewhere and needs to be parapharased*


We’ve all seen tweets with a plethora of spelling mistakes. Our timelines are often filled with hastly sent tweets that are barely legible at times.

In that regard, spelling correction is a useful pre-processing step because this also will help us in reducing multiple copies of words. For example, “Analytics” and “analytcs” will be treated as different words even if they are used in the same sense.

To achieve this we will use the textblob library.

In [ ]:
from textblob import TextBlob
#Example sentence
sentence = TextBlob("I havv goood speling!")
print(sentence.correct())

I have good spelling!


In [ ]:
# Example sentence related to tourism context
print(TextBlob('In Chicago, a lakeside is a goodf lcation to relax').correct())
print(TextBlob('Minesota is briming with natual and cltural bauty').correct()) 

In Chicago, a lakeside is a good location to relax
Minnesota is brimming with natural and cultural beauty


In [ ]:
df.shape

(5880, 46)

In [ ]:
# Apply spell correction on tweets. (This process can take a long)

# Running spell correction on first 10 tweets can give us an idea about how much it will take
# Our dataset has 5880 tweets right now. We can use consider the ratio for execution time
%time df['tweet'][:10].apply(lambda x: str(TextBlob(x).correct())) 

CPU times: user 43 s, sys: 83.1 ms, total: 43.1 s
Wall time: 43.2 s


0     tropical treasure cherish forever exclusive # ...
1     backpack # cuba budget # travelsomeday # armch...
3     experience burns air like local # travelsomeda...
4     can\'t wait grenade sail festival 2021 ? ! exc...
5     best hostel barcelona – fabrizzio ’ s petit re...
6     discover ancient history karay complex labor e...
7     discover ancient history karay complex labor e...
8     discover ancient history karay complex labor e...
9     get know big easy house , virtual tour , music...
10    discover ancient history karay complex labor e...
Name: tweet, dtype: object

In [ ]:
# Apply spell checking on all tweets

%time df['tweet'] = df['tweet'].apply(lambda x: str(TextBlob(x).correct())) 
# Note:if you are only playing with the dataset, you can you to skip this part because of long execution-time
# This part can take 6-7 hours depending on your computer

CPU times: user 4h 38min 25s, sys: 32.7 s, total: 4h 38min 58s
Wall time: 4h 39min 28s


In [ ]:
df.tweet.head()

0    tropical treasure cherish forever exclusive # ...
1    backpack # cuba budget # travelsomeday # armch...
3    experience burns air like local # travelsomeda...
4    can\'t wait grenade sail festival 2021 ? ! exc...
5    best hostel barcelona – fabrizzio ’ s petit re...
Name: tweet, dtype: object

### **Removing Punctuation**

A reminder: if you remove punctuation earlier then html tags and URLs, then it won't be possible to remove them probably later. That's why, removing punctuation the last is a wise choice.


In [ ]:
df['tweet'] = df['tweet'].str.replace('[^\w\s]','')
df['tweet'].head()

0    tropical treasure cherish forever exclusive  t...
1    backpack  cuba budget  travelsomeday  armchair...
3    experience burns air like local  travelsomeday...
4    cant wait grenade sail festival 2021   exclusi...
5    best hostel barcelona  fabrizzio  s petit revi...
Name: tweet, dtype: object

In [ ]:
# Example
sentence = "Do you listen to music a lot?"
re.sub(r'[^\w\s]','',sentence)

'Do you listen to music a lot'

### **TF_IDF**


In [ ]:
#import the TfidfVectorizer from Scikit-Learn.
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=.80, min_df=5, stop_words=None, use_idf=True, norm=None)
transformed_tweets = vectorizer.fit_transform(df.tweet)

print(transformed_tweets.shape)

(5880, 1969)


In [ ]:
dense = transformed_tweets.todense()
denselist = dense.tolist()
feature_names = vectorizer.get_feature_names()
df2 = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
df2.head()

,10,100,1000,11,12,13,14,15,16,17,176,180books,19,1partner,20,200,2019,2020,2021,2022,20second,21,24,25,27,2dancingpandas,2snowy,30,33,40,48,500,5step,80,_sundaysunsets_,aaatravel,abbey,abergavenny,abfabtravels,able,...,without,woman,women,wonder,wonderful,wont,woodland,work,workaway,worktrade,world,worldpackers,worst,worth,would,wouldnt,wreck,write,writer,writers,xeniakaepernick,xochimilco,xunantunich,year,yearly,years,yes,yet,york,yorkshire,yosemite,you,youk,your,yourcruisegurl,yourewelcome,yucatan,yuri,zealand,ziplining
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.040425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
transformed_tweets_as_array = transformed_tweets.toarray()
# use this line of code to verify that the numpy array represents the same number of documents that we have in the file list
len(transformed_tweets_as_array)

5880

### **Vectorizer**

Vectorier is a function for converting tokens into numerical columns. 

We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.



In [ ]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
data_cv = cv.fit_transform(df.tweet)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = df.index
data_dtm


,02038725000,10,100,1000,1000s,1001,104,105,10day,11,12,125,13,137,14,15,16,1600s,17,1700mile,1700s,176,17miledrive,18,180books,1889,1890s,19,1900s,1992,1partner,20,200,2013,2016,2017,2018,2019,2020,2021,...,years,yellow,yellowsprings,yes,yet,york,yorkers,yorkshire,yosemite,yosemitenps,you,youk,young,youngadventures,your,yourcruisegurl,yourewelcome,yours,yous,youtubecommuni3,youtuber,yoyobob,yoyojohn,yt_twitchgrowth,yucatan,yukti77,yuri,zdeněk,zealand,zen,zinc,zionnps,ziplining,zone,zonetraveling,zwettl,zygoma,अरकवल,सof,ফর_আসবofমর_পডয
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6023,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6024,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6025,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
cv = CountVectorizer(max_df = 0.90, min_df= 0.001)
data_cv = cv.fit_transform(df.tweet)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names()) #Document term matrix
data_dtm.index = df.index
data_dtm

,10,100,1000,11,12,14,15,17,180books,19,1partner,20,200,2019,2020,2021,20second,21,24,25,27,2dancingpandas,2snowy,30,33,48,500,5step,80,_sundaysunsets_,aaatravel,abbey,abergavenny,abfabtravels,able,abroad,abudhabi,acatenango,accommodation,accord,...,wildlifephotography,wildlifewednesday,will,wine,winetraveleats,winter,wish,with,without,woman,women,wonder,wonderful,wont,work,workaway,world,worldpackers,worst,worth,would,wouldnt,wreck,write,writer,xeniakaepernick,xunantunich,year,years,yes,york,yorkshire,yosemite,you,your,yourcruisegurl,yourewelcome,yucatan,zealand,ziplining
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6023,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6024,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6025,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Let's pickle it for later use
data_dtm.to_pickle("cleaned_tweets.pkl")

#Notice that the pickled data contains only tweets, not other columns.



**What Can You Do With pickle?**

Pickling is useful for applications where you need some degree of persistency in your data. 
Your program's state data can be saved to disk, so you can continue working on it later on.
It can also be used to send data over a Transmission Control Protocol (TCP) or 
socket connection, or to store python objects in a database. Pickle is very 
useful for when you're working with machine learning algorithms, where you want
to save them to be able to make new predictions at a later time, without
having to rewrite everything or train the model all over again.

Source:https://www.datacamp.com/community/tutorials/pickle-python-tutorial#whatfor




---
# **FEATURE EXTRACTION**

For the feature extraction, we will apply following steps in order. Depending on your study and data, you may skip some of these features and add your own feature extractions.

 word count: counts the number of tokens in the text (separated by a space)

1.   **character count**: 
2. **average word length**


Features that can only be obtained before the text cleaning


* Number of stop words
* The number of punctuation
* Number of hashtag characters
* Number of numerical characters
* Number of Uppercase words
* sentence count

In [ ]:
#Let's save cleaned data

df.to_csv('tourism_tweets_cleaned.csv', index=False)

### **Checking requirements for this notebook**

In [ ]:
!pip3 freeze > requirements.txt

In [ ]:
# show the file's contents
! cat requirements.txt

In [ ]:
# Import the os module
import os

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Print the type of the returned object
print("os.getcwd() returns an object of type: {0}".format(type(cwd)))

Current working directory: /content
os.getcwd() returns an object of type: <class 'str'>
